In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import re


In [4]:
data = pd.read_csv('/mnt/c/Users/AdamC/Plocha/Skola/bachelors_thesis/data/wiki/train.csv')

In [2]:
data = pd.read_csv('/mnt/c/Users/AdamC/Plocha/Skola/bachelors_thesis/data/wiki/test.csv')

In [5]:
data["non_toxic"] = data.iloc[:,2:8].apply(lambda x: 1 if (sum(x)==0) else 0, axis=1)
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,class_binary
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,1
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,1
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,1
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1


In [3]:
cleaned_data = data.copy()

# Removing Hyperlinks from text
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(lambda x: re.sub(r"https?://\S+|www\.\S+", "", x))
# Rmoving emojis from text
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(lambda x: re.sub("["
                                                                                 u"\U0001F600-\U0001F64F"
                                                                                 u"\U0001F300-\U0001F5FF"
                                                                                 u"\U0001F680-\U0001F6FF"
                                                                                 u"\U0001F1E0-\U0001F1FF"
                                                                                 u"\U00002702-\U000027B0"
                                                                                 u"\U000024C2-\U0001F251"
                                                                                 "]+", "", x, flags=re.UNICODE))
# Removing IP addresses from text
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(
    lambda x: re.sub(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", "", x))

In [4]:
# Removing html tags from text
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(lambda x: re.sub(r"<.*?>", "", x))
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(
    lambda x: re.sub(r"\"\"", "\"", x))  # replacing "" with "
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(
    lambda x: re.sub(r"^\"", "", x))  # removing quotation from start and the end of the string
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(lambda x: re.sub(r"\"$", "", x))
# Removing Punctuation / Special characters (;:'".?@!%&*+) which appears more than twice in the text
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(
    lambda x: re.sub(r"[^a-zA-Z0-9\s][^a-zA-Z0-9\s]+", " ", x))
# Removing Special characters
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(lambda x: re.sub(r"[^a-zA-Z0-9\s\"\',:;?!.()]", " ", x))
# Removing extra spaces in text
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(lambda x: re.sub(r"\s\s+", " ", x))
cleaned_data.reset_index(inplace=True)
cleaned_data.drop(columns=["index"], inplace=True)


cleaned_data.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,"From RfC The title is fine as it is, IMO."
2,00013b17ad220c46,Sources Zawe Ashton on Lapland
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [8]:
cleaned_data.to_csv(
    '/mnt/c/Users/AdamC/Plocha/Skola/bachelors_thesis/data/wiki/preprocessed/train.csv')

In [5]:
cleaned_data.to_csv(
    '/mnt/c/Users/AdamC/Plocha/Skola/bachelors_thesis/data/wiki/preprocessed/test.csv')

In [6]:
cleaned_data.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,"From RfC The title is fine as it is, IMO."
2,00013b17ad220c46,Sources Zawe Ashton on Lapland
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [12]:
data = cleaned_data.copy()
C_indexs = data[data["non_toxic"]==1].index

toxic_count = data[data["non_toxic"]==0].shape[0]
drop_indxs = np.random.choice(C_indexs, size=data[data["non_toxic"]==1].shape[0] - toxic_count, replace=False)

# Randomly selecting and dropping non_toxic comments from datasets and leaving 5000 behind
data.drop(drop_indxs, inplace=True)

data.to_csv("/mnt/c/Users/AdamC/Plocha/Skola/bachelors_thesis/data/wiki/preprocessed/train_50_50.csv")
print(data.shape)

(32450, 9)


In [7]:
data = pd.read_csv('/mnt/c/Users/AdamC/Plocha/Skola/bachelors_thesis/submission.csv')

In [8]:
data.head()

,Unnamed: 0,id,toxic,severe_toxic,threat,insult,obscene,identity_hate
0,0,00001cee341fdb12,1.000010,0.201823,0.991163,0.061886,0.943358,0.191943
1,1,0000247867823ef7,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
2,2,00013b17ad220c46,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
3,3,00017563c3f7919a,0.000546,0.000546,0.001109,0.000390,0.002811,0.000742
4,4,00017695ad8997eb,0.002262,0.000010,0.000010,0.000010,0.000010,0.000010


In [9]:
data.columns

Index(['Unnamed: 0', 'id', 'toxic', 'severe_toxic', 'threat', 'insult',
       'obscene', 'identity_hate'],
      dtype='object')

In [17]:
data = data.drop(columns=['Unnamed: 0'])

In [19]:
data.to_csv("/mnt/c/Users/AdamC/Plocha/Skola/bachelors_thesis/submission1.csv", index=False)

In [18]:
data.head()

,id,toxic,severe_toxic,threat,insult,obscene,identity_hate
0,00001cee341fdb12,1.000010,0.201823,0.991163,0.061886,0.943358,0.191943
1,0000247867823ef7,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
2,00013b17ad220c46,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
3,00017563c3f7919a,0.000546,0.000546,0.001109,0.000390,0.002811,0.000742
4,00017695ad8997eb,0.002262,0.000010,0.000010,0.000010,0.000010,0.000010


In [29]:
test_data = pd.read_csv('/mnt/c/Users/AdamC/Plocha/Skola/bachelors_thesis/data/wiki/preprocessed/test.csv')

In [30]:
test_data.head(20)

,Unnamed: 0,id,comment_text
0,0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,1,0000247867823ef7,"From RfC The title is fine as it is, IMO."
2,2,00013b17ad220c46,Sources Zawe Ashton on Lapland
3,3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,4,00017695ad8997eb,I don't anonymously edit articles at all.
5,5,0001ea8717f6de06,Thank you for understanding. I think very high...
6,6,00024115d4cbde0f,Please do not add nonsense to Wikipedia. Such ...
7,7,000247e83dcc1211,:Dear god this site is horrible.
8,8,00025358d4737918,Only a fool can believe in such numbers. The ...
9,9,00026d1092fe71cc,Double Redirects When fixing double redirects...


In [25]:
data[data['toxic'] != -1].to_csv('/mnt/c/Users/AdamC/Plocha/Skola/bachelors_thesis/data/wiki/ground_truth.csv', index=False)

In [26]:
test_labels = data[data['toxic'] != -1]

In [31]:
test_labels.head(20)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,0,0,0,0,0,0
7,000247e83dcc1211,0,0,0,0,0,0
11,0002f87b16116a7f,0,0,0,0,0,0
13,0003e1cccfd5a40a,0,0,0,0,0,0
14,00059ace3e3e9a53,0,0,0,0,0,0
16,000663aff0fffc80,0,0,0,0,0,0
17,000689dd34e20979,0,0,0,0,0,0
19,000844b52dee5f3f,0,0,0,0,0,0
21,00091c35fa9d0465,1,0,0,0,0,0
22,000968ce11f5ee34,0,0,0,0,0,0


In [38]:
print(test_labels.index)

Int64Index([     5,      7,     11,     13,     14,     16,     17,     19,
                21,     22,
            ...
            153142, 153144, 153145, 153147, 153149, 153150, 153151, 153154,
            153155, 153156],
           dtype='int64', length=63978)


In [40]:
test_data_to_save = test_data.iloc[test_labels.index,:]

In [44]:
test_data_to_save.to_csv('/mnt/c/Users/AdamC/Plocha/Skola/bachelors_thesis/data/wiki/preprocessed/test_kaggle.csv', index=False)

In [45]:
data.index

RangeIndex(start=0, stop=153164, step=1)